In [ ]:
# ======================================
# MaskGuard Pro v1.0 - Real-time Mask Detection
# Author: Sham (with Sensei guidance 🥋)
# ======================================

import cv2
import datetime
import csv
import os
from ultralytics import YOLO

# -------------------------------
# INITIAL SETUP
from ultralytics import YOLO

model = YOLO(r"C:\Users\shamm\Downloads\best.pt")
 # Replace with your mask model (e.g., 'best.pt')
cap = cv2.VideoCapture(0)

# Create log file if not exist
if not os.path.exists("mask_log.csv"):
    with open("mask_log.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Mask_Status", "Confidence", "Faces_Detected"])

# -------------------------------
# DRAW HUD FUNCTION
# -------------------------------
def draw_HUD(frame, status, conf, faces):
    h, w, _ = frame.shape
    color = (0,255,0) if status == "Mask" else (0,0,255)
    cv2.putText(frame, f"Status: {status}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
    cv2.putText(frame, f"Confidence: {conf:.2f}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    cv2.putText(frame, f"Faces Detected: {faces}", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    cv2.rectangle(frame, (0,0), (w-1,h-1), color, 4)

# -------------------------------
# MAIN LOOP
# -------------------------------
while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    results = model.predict(frame, stream=True, imgsz=640, conf=0.5)

    faces = 0
    mask_status = "No Face"
    conf_val = 0

    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Draw bounding boxes
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

            faces += 1
            mask_status = model.names[cls]
            conf_val = conf

    # Draw HUD
    draw_HUD(frame, mask_status, conf_val, faces)

    # Log result
    with open("mask_log.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                         mask_status, round(conf_val,2), faces])

    cv2.imshow("MaskGuard Pro", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
